In [2]:
!pip install emoji
!pip install tensorflow
!pip install transformers

In [3]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

model = TFAutoModelForSequenceClassification.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at rabindralamsal/finetuned-bertweet-sentiment-analysis.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [5]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
link = 'https://drive.google.com/file/d/17-X8-Fg4B11bqxt_fZ5B-X2WzeuRky0f/view?usp=sharing' 

In [7]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Train_Feb.csv') 
 
df = pd.read_csv('Train_Feb.csv')
print(df)

                                                      text
0        @k_c12345 Brother this called link 👇\nhttps://...
1        Ukraine invasion: Who is President Volodymyr Z...
2        So it now looks like we're in a new phase of t...
3        @vanderbellen @karlnehammer TOP RUSSIAN AGGRES...
4        @ZelenskyyUa @EmmanuelMacron Ukraine won't fal...
...                                                    ...
1782546  @UNESCO Dear UNESCO. I'm a peaceful citizen of...
1782547  This is not Ukraine vs Russia war. This is a c...
1782548  Russia 🇷🇺 is warn... Don't provoke the 4nation...
1782549  Africa Report: How Africa factors in #RussiaUk...
1782550  @BanteSaket @LukeMackay13 @Reporter_indian I l...

[1782551 rows x 1 columns]


In [8]:
df['text'] = df['text'].astype('str') 

In [ ]:
#df_sample = df.sample(n=50000, random_state=2)

In [ ]:
#df_sample

In [13]:
docs = list(df.loc[:, "text"].values)

In [14]:
#get index for splitting dataset at index: 178255
from math import floor
floor(len(docs) / 10)


178255

In [15]:
docs = docs[:178255]

In [16]:
from tqdm import tqdm
scores = []

for i in tqdm(range(len(docs))):
    input = tokenizer.encode(docs[i], return_tensors="tf")
    output = model.predict(input)[0]
    prediction = tf.nn.softmax(output, axis=1).numpy()
    sentiment = np.argmax(prediction)
    
    scores.append({"Label": sentiment,
                       "Negative": prediction[0,0],
                       "Neutral": prediction[0,1],
                       "Positive": prediction[0,2]
                  })

100%|██████████| 178255/178255 [3:12:18<00:00, 15.45it/s]


In [17]:
df = df[:178255]

In [18]:
sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score)

In [19]:
df

,text,Label,Negative,Neutral,Positive
0,@k_c12345 Brother this called link 👇\nhttps://...,1,0.335056,0.656753,0.008192
1,Ukraine invasion: Who is President Volodymyr Z...,1,0.272589,0.721097,0.006314
2,So it now looks like we're in a new phase of t...,0,0.864383,0.132682,0.002935
3,@vanderbellen @karlnehammer TOP RUSSIAN AGGRES...,0,0.837160,0.149967,0.012874
4,@ZelenskyyUa @EmmanuelMacron Ukraine won't fal...,1,0.015998,0.844986,0.139016
...,...,...,...,...,...
178250,"US, NATO seek to shore up defenses as Russia-U...",1,0.009232,0.971955,0.018813
178251,@KenHolmes10 @graceblakeley @zarahsultana The ...,0,0.959561,0.037926,0.002513
178252,@Rajeev_vajeer @MichaelaNewell_ @ZelenskyyUa @...,1,0.155604,0.835924,0.008472
178253,@culturisk America can't do shit rn because Uk...,0,0.918778,0.078283,0.002939


In [21]:
from google.colab import drive
drive.mount('drive')

df.to_csv('finetuned_Bertweet_SA_Feb_Train_p1.csv')
!cp finetuned_Bertweet_SA_Feb_Train_p1.csv 'drive/My Drive/Colab Notebooks'

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
